<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_mask2former_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR MASK2FORMER
================================================================================
Sistema de evaluación para el modelo Mask2Former con estructura estandarizada
para comparación de modelos de segmentación de instancias.

Funcionalidades:
- Procesamiento incremental con checkpoint
- Gestión óptima de memoria para Colab gratuito
- Guardado automático después de cada imagen
- Generación de visualizaciones
- Máscaras organizadas por fotografía
- JSON con métricas de rendimiento

Entrada: Imágenes desde /TFM/0_Imagenes/ o dataset COCO
Salida: JSON, máscaras NPZ y visualizaciones en /TFM/2_Modelos/mask2former/

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña
Fecha: 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR MASK2FORMER\n================================================================================\nSistema de evaluación para el modelo Mask2Former con estructura estandarizada\npara comparación de modelos de segmentación de instancias.\n\nFuncionalidades:\n- Procesamiento incremental con checkpoint\n- Gestión óptima de memoria para Colab gratuito\n- Guardado automático después de cada imagen\n- Generación de visualizaciones\n- Máscaras organizadas por fotografía\n- JSON con métricas de rendimiento\n\nEntrada: Imágenes desde /TFM/0_Imagenes/ o dataset COCO\nSalida: JSON, máscaras NPZ y visualizaciones en /TFM/2_Modelos/mask2former/\n\nAutor: Jesús L.\nProyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación\nUniversidad: Universidad Oberta de Cataluña\nFecha: 2025\n================================================================================\n'

In [29]:
import os
import gc
import time
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

warnings.filterwarnings('ignore')

print("Librerías cargadas correctamente")

Librerías cargadas correctamente


In [30]:
# =============================================================================
# CONFIGURACIÓN
# =============================================================================

@dataclass
class ModeloMask2Former:
    """Información de un modelo Mask2Former"""
    nombre: str
    nombre_corto: str
    hf_id: str
    tipo: str
    dataset: str
    backbone: str

MODELOS_DISPONIBLES = {
    'large_coco': ModeloMask2Former(
        nombre='Swin-Large COCO Instance',
        nombre_corto='large_coco',
        hf_id='facebook/mask2former-swin-large-coco-instance',
        tipo='instance',
        dataset='COCO',
        backbone='Swin-Large'
    ),
    'base_coco': ModeloMask2Former(
        nombre='Swin-Base COCO Instance',
        nombre_corto='base_coco',
        hf_id='facebook/mask2former-swin-base-coco-instance',
        tipo='instance',
        dataset='COCO',
        backbone='Swin-Base'
    ),
    'tiny_coco': ModeloMask2Former(
        nombre='Swin-Tiny COCO Instance',
        nombre_corto='tiny_coco',
        hf_id='facebook/mask2former-swin-tiny-coco-instance',
        tipo='instance',
        dataset='COCO',
        backbone='Swin-Tiny'
    )
    # 'large_ade': ModeloMask2Former(
    #     nombre='Swin-Large ADE20K Semantic',
    #     nombre_corto='large_ade',
    #     hf_id='facebook/mask2former-swin-large-ade-semantic',
    #     tipo='semantic',
    #     dataset='ADE20K',
    #     backbone='Swin-Large'
    # )
}

@dataclass
class ConfiguracionUmbrales:
    """Configuración de umbrales de confianza"""
    nombre: str
    valores: List[float]
    descripcion: str

CONFIGURACIONES_UMBRALES = {
    'maxima_sensibilidad': ConfiguracionUmbrales(
        nombre='maxima_sensibilidad',
        valores=[0.0001, 0.001, 0.01, 0.1],
        descripcion='Detecta todas las posibles personas'
    ),
    'alta_sensibilidad': ConfiguracionUmbrales(
        nombre='alta_sensibilidad',
        valores=[0.001, 0.01, 0.05, 0.1, 0.3],
        descripcion='Balance hacia detección temprana'
    ),
    'media_sensibilidad': ConfiguracionUmbrales(
        nombre='media_sensibilidad',
        valores=[0.01, 0.1, 0.3, 0.5],
        descripcion='Balance entre precisión y recall'
    ),
    'baja_sensibilidad': ConfiguracionUmbrales(
        nombre='baja_sensibilidad',
        valores=[0.3, 0.5, 0.7],
        descripcion='Solo detecciones muy confiables'
    ),
     'experimental_coco': ConfiguracionUmbrales(
        nombre='experimental_coco',
        valores=[0.1, 0.3, 0.5, 0.7, 0.9],
        descripcion='Rango experimental para modelos COCO instance'
    )
}

@dataclass
class ConfiguracionEvaluacion:
    """Configuración principal del sistema"""
    # Modelos a evaluar
    modelos_evaluar: List[str] = None  # None = todos

    # Configuraciones de umbrales a evaluar
    configs_umbrales: List[str] = None  # None = ['media_sensibilidad']

    # configs_umbrales: List[str] = field(default_factory=lambda: ['experimental_coco'])

    # Rutas
    ruta_base: Path = Path("/content/drive/MyDrive/TFM")
    ruta_imagenes: Path = None
    ruta_resultados: Path = None

    # Procesamiento
    max_dimension: int = 1024
    pausa_entre_imagenes: float = 2.0
    pausa_entre_modelos: float = 5.0

    def __post_init__(self):
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "2_Modelos" / "mask2former"
        if self.modelos_evaluar is None:
            self.modelos_evaluar = list(MODELOS_DISPONIBLES.keys())
        if self.configs_umbrales is None:
            self.configs_umbrales = ['media_sensibilidad']

In [8]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares"""

    @staticmethod
    def cargar_imagen(ruta: Path, max_dim: int = None) -> Tuple[Image.Image, Dict]:
        """Carga imagen con opción de redimensionar"""
        img = Image.open(ruta).convert("RGB")
        dim_orig = img.size

        redim = False
        if max_dim and max(img.size) > max_dim:
            ratio = max_dim / max(img.size)
            new_size = tuple(int(d * ratio) for d in img.size)
            img = img.resize(new_size, Image.LANCZOS)
            redim = True

        info = {
            'original': {'width': dim_orig[0], 'height': dim_orig[1]},
            'procesada': {'width': img.size[0], 'height': img.size[1], 'redimensionada': redim}
        }
        return img, info

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """Guarda JSON"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)

    @staticmethod
    def guardar_mascara_npz(mascara: np.ndarray, ruta: Path):
        """Guarda máscara en NPZ comprimido"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        np.savez_compressed(ruta, mascara=mascara.astype(np.float32))

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def obtener_memoria_gpu() -> Dict:
        """Info de memoria GPU"""
        if not torch.cuda.is_available():
            return {'disponible': False}
        return {
            'disponible': True,
            'nombre': torch.cuda.get_device_name(0),
            'asignada_mb': round(torch.cuda.memory_allocated(0) / 1024**2, 2),
            'reservada_mb': round(torch.cuda.memory_reserved(0) / 1024**2, 2)
        }

In [9]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """Maneja checkpoint para retomar procesamiento"""

    def __init__(self, ruta: Path):
        self.ruta = ruta
        self.datos = self._cargar()

    def _cargar(self) -> Dict:
        if self.ruta.exists():
            with open(self.ruta, 'r') as f:
                return json.load(f)
        return {
            'completadas': [],
            'timestamp': datetime.now().isoformat()
        }

    def guardar(self):
        self.ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(self.ruta, 'w') as f:
            json.dump(self.datos, f, indent=2)

    def marcar_completada(self, nombre: str):
        if nombre not in self.datos['completadas']:
            self.datos['completadas'].append(nombre)
            self.guardar()

    def obtener_pendientes(self, todas: List[str]) -> List[str]:
        return [img for img in todas if img not in self.datos['completadas']]

In [10]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """Genera visualizaciones estilo SAM2/YOLO"""

    @staticmethod
    def generar_visualizacion(
        imagen: np.ndarray,
        mascaras: List[np.ndarray],
        scores: List[float],
        ruta_salida: Path,
        titulo: str = "Detección"
    ):
        """Genera visualización simple con máscaras coloreadas"""
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        # Original
        axes[0].imshow(imagen)
        axes[0].set_title('Imagen Original', fontsize=14)
        axes[0].axis('off')

        # Con máscaras
        axes[1].imshow(imagen)

        if len(mascaras) > 0:
            # Colores para cada máscara
            colors = plt.cm.tab20(np.linspace(0, 1, len(mascaras)))

            for idx, (mascara, score) in enumerate(zip(mascaras, scores)):
                # Overlay de máscara
                color_mask = np.zeros((*mascara.shape, 4))
                color_mask[mascara > 0.5] = [*colors[idx][:3], 0.5]
                axes[1].imshow(color_mask)

                # Bounding box
                coords = np.where(mascara > 0.5)
                if len(coords[0]) > 0:
                    y_min, y_max = coords[0].min(), coords[0].max()
                    x_min, x_max = coords[1].min(), coords[1].max()

                    rect = mpatches.Rectangle(
                        (x_min, y_min),
                        x_max - x_min,
                        y_max - y_min,
                        linewidth=2,
                        edgecolor=colors[idx],
                        facecolor='none'
                    )
                    axes[1].add_patch(rect)

                    # Label
                    axes[1].text(
                        x_min, y_min - 5,
                        f'P{idx+1}: {score:.2f}',
                        color='white',
                        fontsize=10,
                        bbox=dict(boxstyle='round', facecolor=colors[idx], alpha=0.8)
                    )

        axes[1].set_title(f'{titulo}: {len(mascaras)} persona(s)', fontsize=14)
        axes[1].axis('off')

        plt.tight_layout()
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

In [34]:
# =============================================================================
# PROCESADOR MASK2FORMER
# =============================================================================

class ProcesadorMask2Former:
    """Procesador para modelos Mask2Former"""

    def __init__(self, modelo_info: ModeloMask2Former, config: ConfiguracionEvaluacion):
        self.modelo_info = modelo_info
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"  Cargando: {modelo_info.nombre}")
        print(f"  Device: {self.device}")

        self.processor = AutoImageProcessor.from_pretrained(modelo_info.hf_id)
        self.model = Mask2FormerForUniversalSegmentation.from_pretrained(modelo_info.hf_id)
        self.model.to(self.device)
        self.model.eval()

        # Verificar configuracion de clases
        self._verificar_configuracion_clases()

        print(f"  Modelo cargado correctamente")

    def _verificar_configuracion_clases(self):
        """Verifica y muestra la configuracion de clases del modelo"""
        print(f"\n  === CONFIGURACION DE CLASES ===")

        if hasattr(self.model.config, 'id2label'):
            # Buscar clase "person"
            clase_persona = None
            for clase_id, nombre in self.model.config.id2label.items():
                if any(term in nombre.lower() for term in ['person', 'people', 'human']):
                    clase_persona = clase_id
                    print(f"  Clase PERSON encontrada: ID={clase_id}, Nombre='{nombre}'")
                    break

            if clase_persona is None:
                print(f"  WARNING: No se encontro clase 'person' explicita")
                print(f"  Asumiendo clase 0. Clases disponibles:")
                for cid, nombre in list(self.model.config.id2label.items())[:10]:
                    print(f"    - ID {cid}: {nombre}")
        else:
            print(f"  WARNING: Modelo sin id2label, usando clase 0 por defecto")

        print(f"  ================================\n")

    def procesar_imagen(
        self,
        ruta_imagen: Path,
        umbrales: List[float],
        ruta_resultados: Path
    ) -> Dict:
        """Procesa una imagen con todos los umbrales"""

        nombre_foto = ruta_imagen.stem
        timestamp_inicio = time.time()

        # Cargar imagen
        img_pil, info_dim = Utilidades.cargar_imagen(ruta_imagen, self.config.max_dimension)
        img_array = np.array(img_pil)

        # Info GPU inicial
        gpu_ini = Utilidades.obtener_memoria_gpu()

        # Inferencia
        t_inf_ini = time.time()
        inputs = self.processor(images=img_pil, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)

        tiempo_inferencia = (time.time() - t_inf_ini) * 1000

        # GPU pico
        gpu_pico = Utilidades.obtener_memoria_gpu()

        # Procesar por umbral
        resultados_umbrales = {}

        for umbral in umbrales:
            resultado_umbral = self._procesar_umbral(
                outputs, img_pil, umbral, nombre_foto, ruta_resultados
            )
            resultados_umbrales[f'umbral_{umbral}'] = resultado_umbral

        # GPU final
        gpu_fin = Utilidades.obtener_memoria_gpu()

        tiempo_total = (time.time() - timestamp_inicio) * 1000

        # Construir resultado completo
        resultado = {
            'metadata': {
                'modelo': self.modelo_info.nombre,
                'modelo_tipo': self.modelo_info.tipo,
                'modelo_dataset': self.modelo_info.dataset,
                'modelo_backbone': self.modelo_info.backbone,
                'nombre_foto': ruta_imagen.name,
                'timestamp': datetime.now().isoformat(),
                'dimensiones': info_dim
            },
            'rendimiento': {
                'tiempo_inferencia_ms': round(tiempo_inferencia, 2),
                'tiempo_total_ms': round(tiempo_total, 2),
                'gpu_inicial_mb': gpu_ini.get('asignada_mb', 0),
                'gpu_pico_mb': gpu_pico.get('asignada_mb', 0),
                'gpu_final_mb': gpu_fin.get('asignada_mb', 0),
                'dispositivo': gpu_ini.get('nombre', 'CPU')
            },
            'detecciones_por_umbral': resultados_umbrales
        }

        return resultado

    def _procesar_umbral(
        self,
        outputs,
        img_pil: Image.Image,
        umbral: float,
        nombre_foto: str,
        ruta_resultados: Path
    ) -> Dict:
        """Procesa un umbral especifico"""

        target_size = img_pil.size[::-1]

        # Post-procesamiento segun tipo de modelo
        if self.modelo_info.tipo == 'instance':
            # METODO 1: Intentar segmentacion panoptica (mas compatible con Mask2Former)
            try:
                results = self.processor.post_process_panoptic_segmentation(
                    outputs,
                    target_sizes=[target_size],
                    threshold=umbral,
                    mask_threshold=0.45,
                    overlap_mask_area_threshold=0.7
                )[0]

                segments_info = results.get("segments_info", [])
                segmentation = results.get("segmentation", None)

                print(f"    [DEBUG Umbral {umbral}] Metodo panoptico - Segmentos: {len(segments_info)}")

                if len(segments_info) > 0 and segmentation is not None:
                    # Filtrar personas (clase 0 en COCO)
                    indices_persona = [i for i, seg in enumerate(segments_info) if seg['label_id'] == 0]
                    num_personas = len(indices_persona)

                    mascaras_personas = []
                    scores_personas = []

                    if num_personas > 0:
                        segmentation_np = segmentation.cpu().numpy()
                        for idx in indices_persona:
                            seg_id = segments_info[idx]['id']
                            mascara = (segmentation_np == seg_id).astype(np.uint8)
                            mascaras_personas.append(mascara)
                            scores_personas.append(segments_info[idx].get('score', 1.0))

                        print(f"    [DEBUG] Personas encontradas: {num_personas}")
                    else:
                        print(f"    [DEBUG] No se encontraron personas")
                        if len(segments_info) > 0:
                            clases_unicas = list(set([seg['label_id'] for seg in segments_info]))
                            print(f"    [DEBUG] Otras clases: {clases_unicas}")
                else:
                    # Si panoptico no devuelve resultados, intentar instance
                    raise ValueError("Panoptic segmentation retorno vacio, intentando instance")

            except Exception as e:
                # METODO 2: Fallback a segmentacion de instancia
                print(f"    [DEBUG] Panoptico fallo, usando instance segmentation")

                results = self.processor.post_process_instance_segmentation(
                    outputs,
                    target_sizes=[target_size],
                    threshold=umbral,
                    mask_threshold=0.5,
                    overlap_mask_area_threshold=0.8
                )[0]

                labels = results.get("labels", torch.tensor([]))
                scores = results.get("scores", torch.tensor([]))
                masks = results.get("masks", torch.tensor([]))

                print(f"    [DEBUG Umbral {umbral}] Total detecciones: {len(labels)}")

                if len(labels) > 0:
                    print(f"    [DEBUG] Clases: {[int(l.item()) for l in labels]}")
                    print(f"    [DEBUG] Scores: {[round(float(s.item()), 3) for s in scores]}")

                # Filtrar personas (clase 0 en COCO)
                indices_persona = [i for i, l in enumerate(labels) if int(l.item()) == 0]
                num_personas = len(indices_persona)

                if num_personas > 0:
                    print(f"    [DEBUG] Personas encontradas: {num_personas}")
                else:
                    print(f"    [DEBUG] No se encontraron personas")
                    if len(labels) > 0:
                        clases_unicas = list(set([int(l.item()) for l in labels]))
                        print(f"    [DEBUG] Otras clases: {clases_unicas}")

                scores_personas = [float(scores[i].item()) for i in indices_persona]
                mascaras_personas = [masks[i].cpu().numpy() for i in indices_persona]

        elif self.modelo_info.tipo == 'semantic':
            results = self.processor.post_process_semantic_segmentation(
                outputs,
                target_sizes=[target_size]
            )[0]

            segmentation = results.cpu().numpy()

            # En ADE20K, buscar clase persona (tipicamente clase 12)
            persona_mask = (segmentation == 12).astype(np.uint8)

            if persona_mask.sum() > 0:
                num_personas = 1
                scores_personas = [1.0]
                mascaras_personas = [persona_mask]
            else:
                num_personas = 0
                scores_personas = []
                mascaras_personas = []

        else:
            # Tipo de modelo desconocido
            print(f"    [ERROR] Tipo de modelo desconocido: {self.modelo_info.tipo}")
            num_personas = 0
            scores_personas = []
            mascaras_personas = []

        # Guardar mascaras
        dir_mascaras = ruta_resultados / "mascaras" / nombre_foto / f"umbral_{umbral}"
        archivos_mascaras = []

        for idx, mascara in enumerate(mascaras_personas):
            nombre_npz = f"mascara_{idx+1:03d}.npz"
            ruta_npz = dir_mascaras / nombre_npz
            Utilidades.guardar_mascara_npz(mascara, ruta_npz)
            archivos_mascaras.append(str(ruta_npz.relative_to(ruta_resultados)))

        # Informacion del umbral
        return {
            'num_personas_detectadas': num_personas,
            'scores': scores_personas,
            'score_promedio': float(np.mean(scores_personas)) if scores_personas else 0.0,
            'score_maximo': float(max(scores_personas)) if scores_personas else 0.0,
            'archivos_mascaras': archivos_mascaras,
            'directorio_mascaras': str(dir_mascaras.relative_to(ruta_resultados)) if num_personas > 0 else None
        }

    def limpiar_memoria(self):
        """Libera recursos del modelo"""
        del self.model
        del self.processor
        Utilidades.liberar_memoria()

In [24]:
# =============================================================================
# EVALUADOR PRINCIPAL
# =============================================================================

class EvaluadorMask2Former:
    """Evaluador principal que procesa todos los modelos"""

    def __init__(self, config: ConfiguracionEvaluacion):
        self.config = config

    def evaluar_modelo_con_config_umbrales(
        self,
        modelo_key: str,
        config_umbral_nombre: str,
        imagenes: List[Path]
    ) -> Dict:
        """Evalúa un modelo con una configuración de umbrales específica"""

        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        umbrales_config = CONFIGURACIONES_UMBRALES[config_umbral_nombre]

        # Ruta de resultados incluye la configuración de umbrales
        ruta_resultados = (self.config.ruta_resultados /
                          modelo_info.nombre_corto /
                          config_umbral_nombre /
                          "resultados")

        print(f"\n{'='*80}")
        print(f"MODELO: {modelo_info.nombre}")
        print(f"UMBRALES: {umbrales_config.nombre} - {umbrales_config.valores}")
        print(f"{'='*80}")

        # Checkpoint
        checkpoint = GestorCheckpoint(ruta_resultados / "checkpoint.json")
        nombres_imgs = [img.name for img in imagenes]
        pendientes = checkpoint.obtener_pendientes(nombres_imgs)
        imgs_procesar = [img for img in imagenes if img.name in pendientes]

        print(f"Pendientes: {len(imgs_procesar)}/{len(imagenes)}")

        # if len(imgs_procesar) == 0:
        #     print("Todas las imágenes ya procesadas")
        #     return {'procesadas': 0}

        # Cargar modelo
        procesador = ProcesadorMask2Former(modelo_info, self.config)

        # Procesar imágenes
        tiempo_inicio = time.time()

        for i, img_path in enumerate(imgs_procesar, 1):
            print(f"\n[{i}/{len(imgs_procesar)}] {img_path.name}")

            try:
                # Procesar
                resultado = procesador.procesar_imagen(
                    img_path,
                    umbrales_config.valores,
                    ruta_resultados
                )

                # Añadir info de configuración de umbrales
                resultado['metadata']['config_umbrales'] = {
                    'nombre': umbrales_config.nombre,
                    'valores': umbrales_config.valores,
                    'descripcion': umbrales_config.descripcion
                }

                # Guardar JSON
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                nombre_json = f"{img_path.stem}_{timestamp}.json"
                ruta_json = ruta_resultados / "json" / nombre_json
                Utilidades.guardar_json(resultado, ruta_json)

                # Generar visualización (umbral medio)
                umbral_vis = umbrales_config.valores[len(umbrales_config.valores)//2]
                datos_umbral = resultado['detecciones_por_umbral'][f'umbral_{umbral_vis}']

                if datos_umbral['num_personas_detectadas'] > 0:
                    # Cargar máscaras para visualización
                    mascaras_vis = []
                    for archivo_mask in datos_umbral['archivos_mascaras']:
                        ruta_completa = ruta_resultados / archivo_mask
                        data = np.load(ruta_completa)
                        mascaras_vis.append(data['mascara'])

                    img_array = np.array(Image.open(img_path).convert("RGB"))
                    if self.config.max_dimension and max(img_array.shape[:2]) > self.config.max_dimension:
                        ratio = self.config.max_dimension / max(img_array.shape[:2])
                        new_h = int(img_array.shape[0] * ratio)
                        new_w = int(img_array.shape[1] * ratio)
                        img_array = cv2.resize(img_array, (new_w, new_h))

                    nombre_vis = f"{img_path.stem}_{timestamp}.png"
                    ruta_vis = ruta_resultados / "visualizaciones" / nombre_vis

                    GeneradorVisualizaciones.generar_visualizacion(
                        img_array,
                        mascaras_vis,
                        datos_umbral['scores'],
                        ruta_vis,
                        f"{modelo_info.nombre_corto} - {umbrales_config.nombre}"
                    )

                checkpoint.marcar_completada(img_path.name)

                print(f"  Completada - {datos_umbral['num_personas_detectadas']} personas")

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                import traceback
                traceback.print_exc()

            finally:
                Utilidades.liberar_memoria()
                if i < len(imgs_procesar):
                    time.sleep(self.config.pausa_entre_imagenes)

        tiempo_total = time.time() - tiempo_inicio

        # Limpiar modelo
        del procesador
        Utilidades.liberar_memoria()

        print(f"\nConfiguración completada en {tiempo_total:.1f}s")

        return {
            'procesadas': len(imgs_procesar),
            'tiempo_segundos': round(tiempo_total, 2)
        }

    def ejecutar(self):
        """Ejecuta evaluación completa"""

        print(f"\n{'='*80}")
        print("EVALUADOR MASK2FORMER - NOTEBOOK 2")
        print(f"{'='*80}")
        print(f"Modelos: {len(self.config.modelos_evaluar)}")
        print(f"Configuraciones de umbrales: {len(self.config.configs_umbrales)}")
        for config_name in self.config.configs_umbrales:
            umbrales = CONFIGURACIONES_UMBRALES[config_name]
            print(f"  - {config_name}: {umbrales.valores}")
        print(f"Imágenes: {self.config.ruta_imagenes}")

        # Cargar imágenes
        imagenes = sorted(self.config.ruta_imagenes.glob("*.jpg"))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.JPG")))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.png")))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.PNG")))
        imagenes = sorted(set(imagenes))

        print(f"Total imágenes encontradas: {len(imagenes)}")

        if not imagenes:
            print("ERROR: No se encontraron imágenes")
            return

        # Evaluar cada combinación de modelo + configuración de umbrales
        total_combinaciones = len(self.config.modelos_evaluar) * len(self.config.configs_umbrales)
        combinacion_actual = 0

        for modelo_key in self.config.modelos_evaluar:
            for config_umbral in self.config.configs_umbrales:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                self.evaluar_modelo_con_config_umbrales(modelo_key, config_umbral, imagenes)

                if combinacion_actual < total_combinaciones:
                    print(f"\nPausa entre configuraciones: {self.config.pausa_entre_modelos}s")
                    time.sleep(self.config.pausa_entre_modelos)

        print(f"\n{'='*80}")
        print("EVALUACIÓN COMPLETADA")
        print(f"{'='*80}")
        print(f"Resultados en: {self.config.ruta_resultados}")

In [33]:
# =============================================================================
# EJECUCIÓN
# =============================================================================

def main():
    """Función principal"""

    # Montar Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
    except:
        pass

    # Configurar
    config = ConfiguracionEvaluacion(
        # Seleccionar modelos (None = todos los 4)
        modelos_evaluar=None,

        # Seleccionar MÚLTIPLES configuraciones de umbrales
        configs_umbrales=[
            'experimental_coco',
            'baja_sensibilidad',
            'media_sensibilidad',    # [0.01, 0.1, 0.3, 0.5]
            'alta_sensibilidad',     # [0.001, 0.01, 0.05, 0.1, 0.3]
            'maxima_sensibilidad'    # [0.0001, 0.001, 0.01, 0.1]

        ],

        # Opciones
        max_dimension=1024,
        pausa_entre_imagenes=2.0,
        pausa_entre_modelos=5.0
    )

    # Ejecutar
    evaluador = EvaluadorMask2Former(config)
    evaluador.ejecutar()

In [35]:
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

EVALUADOR MASK2FORMER - NOTEBOOK 2
Modelos: 3
Configuraciones de umbrales: 5
  - experimental_coco: [0.1, 0.3, 0.5, 0.7, 0.9]
  - baja_sensibilidad: [0.3, 0.5, 0.7]
  - media_sensibilidad: [0.01, 0.1, 0.3, 0.5]
  - alta_sensibilidad: [0.001, 0.01, 0.05, 0.1, 0.3]
  - maxima_sensibilidad: [0.0001, 0.001, 0.01, 0.1]
Imágenes: /content/drive/MyDrive/TFM/0_Imagenes
Total imágenes encontradas: 20

################################################################################
COMBINACIÓN 1/15
################################################################################

MODELO: Swin-Large COCO Instance
UMBRALES: experimental_coco - [0.1, 0.3, 0.5, 0.7, 0.9]
Pendientes: 20/20
  Cargando: Swin-Large COCO Instance
  Device: cuda

  === CONFIGURACION DE CLASES ===
  Clase PERSON encontrada: ID=0, Nombre='person'

  Modelo cargado correctamente

[1/20] _DSC0023.jp

`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.


    [DEBUG Umbral 0.1] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.3] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.5] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.7] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.9] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
  Completada - 1 personas

[2/20] _DSC0036.jpg


`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.


    [DEBUG Umbral 0.1] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.3] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.5] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.7] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.9] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
  Completada - 1 personas

[3/20] _DSC0071.jpg


...

    [DEBUG Umbral 0.0001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.01] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.1] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
  Completada - 1 personas

[19/20] _DSC0962.jpg


`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.


    [DEBUG Umbral 0.0001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.01] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.1] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
  Completada - 1 personas

[20/20] _DSC0987.jpg


`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.


    [DEBUG Umbral 0.0001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.001] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.01] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
    [DEBUG Umbral 0.1] Metodo panoptico - Segmentos: 1
    [DEBUG] Personas encontradas: 1
  Completada - 1 personas

Configuración completada en 111.3s

EVALUACIÓN COMPLETADA
Resultados en: /content/drive/MyDrive/TFM/2_Modelos/mask2former
